In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('datasets\caracteres-limpo.csv', header=None)

ruido = pd.read_csv('datasets\caracteres-ruido20.csv', header=None)

df

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,-1,-1,1,1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,-1,-1,-1,-1,-1,-1
1,1,1,1,1,1,1,-1,-1,1,-1,...,1,1,-1,-1,1,-1,-1,-1,-1,-1
2,-1,-1,1,1,1,1,1,-1,1,-1,...,1,1,-1,-1,-1,1,-1,-1,-1,-1
3,1,1,1,1,1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,-1,1,-1,-1,-1
4,1,1,1,1,1,1,1,-1,1,-1,...,1,1,1,-1,-1,-1,-1,1,-1,-1
5,-1,-1,-1,1,1,1,1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1
6,1,1,1,-1,-1,1,1,-1,1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
7,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,-1,-1,-1,-1,-1,-1
8,1,1,1,1,1,1,-1,1,-1,-1,...,1,1,-1,-1,1,-1,-1,-1,-1,-1
9,-1,-1,1,1,1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,1,-1,-1,-1,-1


# Hiperparametros

## funções

In [3]:
Activations = {
    'tanh' : (lambda x: np.tanh(x))
    }
D_Activations = {
    'tanh' : (lambda x: 1-x**2)
    }
Errors = {
    'SR' : (lambda x,y: (x-y)**2)
    }
D_Errors = {
    'SR' : (lambda x,y: -2*(x-y))
    }

# Parametros

In [4]:
weights_L1 = [[]]
bias_L1 = []
weights_L2 = [[]]
bias_L2 = []

# implementação do algoritmo

## Inicialização dos pesos e viéses

In [5]:
def start_weights(i, j):
    weights = []
    for _ in range(j):
        temp = []
        for _ in range(i):
            temp.append(random.uniform(min_lim, max_lim)+.01)
        weights.append(temp)
    return np.array(weights)

In [6]:
def start_bias(j):
    bias = []
    for _ in range(j):
        bias.append(random.uniform(min_lim, max_lim)+.01)
    return np.array(bias)

## Feed forward

In [7]:
def feed_forward(x):
    temp = []

    output_L1 = []
    output_L2 = []

    for i,w in enumerate(weights_L1):
        temp = [X*W for (X, W) in zip(x, w)]
        temp = np.sum(temp) + bias_L1[i]
        output_L1.append(activation_func(temp))

    temp = []

    for i,w in enumerate(weights_L2):
        temp = [X*W for (X, W) in zip(output_L1, w)]
        temp = np.sum(temp) + bias_L2[i]
        output_L2.append(activation_func(temp))

    return np.array(output_L1), np.array(output_L2)
    

In [8]:
suco = [1,2,3,4]
fruta = [1,2,3,4]
sujo = 1

oswaldo = [W*X for W,X in zip(suco, fruta)]
print(oswaldo)

macaco = np.sum(oswaldo)
print(macaco)

sagui = macaco + sujo
print(sagui)

[1, 4, 9, 16]
30
31


## calculo do erro total

In [9]:
def error_calculation(y, output_L2):
    total_error = 0
    for i in range(L2):
        SR = error_func(y[i], output_L2[i])
        total_error += SR
    return total_error

## back propagation

In [10]:
def back_propagation(x, y, output_L1, output_L2):
    # gradient descent L2
    delta_L2 = error_deriv(y, output_L2) * activation_deriv(output_L2)

    # updating weights and biases L2
    for i in range(L2):
        for j in range(L1):
            weights_L2[i][j] -= learning_rate * delta_L2[i] * output_L1[j]
            bias_L2[i] -= learning_rate * delta_L2[i]

    # gradient descent L1
    delta_L1 = np.matmul(weights_L2.T, delta_L2) * activation_deriv(output_L1)

    # updating weights and biases L1
    for i in range(L1):
        for j in range(L0):
            weights_L1[i][j] -= learning_rate * delta_L1[i] * x[j]
            bias_L1[i] -= learning_rate * delta_L1[i]
    #print(weights_L2[0])

    

In [11]:
jailson = [[1,2],
           [3,4]]

glauber = [1,2]

jailson = np.array(jailson)

glauber = np.array(glauber)

suzana = np.matmul(jailson, glauber)

print(suzana)

roger = []
jail = [[1,3],
        [2,4]]
jail = np.array(jail)
for j in jail.T:
    roger.append(np.sum(j*glauber))

roger = np.array(roger)

print(roger)

ruga = np.matmul(jail.T, glauber)

print(ruga)

[ 5 11]
[ 5 11]
[ 5 11]


## algoritmo de treinamento do modelo

In [12]:
def fit(X, Y):
    epoch = 0

    while epoch < max_epochs:
        for i in range(len(X.index)):
            x = np.array(X.iloc[i].values)
            y = np.array(Y.iloc[i].values)
            
            # feed forward
            output_L1, output_L2 = feed_forward(x)

            # error calculation
            total_error = error_calculation(y, output_L2)

            # back propagation
            back_propagation(x, y, output_L1, output_L2)
        
        total_error = (total_error / len(X.index))

        epoch += 1

## algoritmo de teste

In [35]:
def output_treatment(output):
    print(output)
    temp = np.empty(len(output))
    max = -1
    for i, o in enumerate(output):
        if o > max:
            max = o
    for i in range(len(temp)):
        if output[i] != max:
            temp[i] = -1
        else:
            temp[i] = 1


    
    return temp



In [14]:
def predict(X):
    preds = []
    

    for i in range(len(X.index)):
        x = np.array(X.iloc[i].values)

        output_L1, output_L2 = feed_forward(x)

        output_L2 = output_treatment(output_L2)

        preds.append(output_L2)

    preds = pd.DataFrame(preds)
    preds = preds.astype('int64')

    return preds

# Uso da IA

## Separação de treino e teste

In [15]:
X = df.iloc[:,0:63]
X_ruido = ruido.iloc[:,0:63]
y_ruido = ruido.iloc[:,63:70]

X.head()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,-1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,1,1,1,-1,1,1,1
1,1,1,1,1,1,1,-1,-1,1,-1,...,-1,-1,1,1,1,1,1,1,1,-1
2,-1,-1,1,1,1,1,1,-1,1,-1,...,-1,-1,1,-1,-1,1,1,1,1,-1
3,1,1,1,1,1,-1,-1,-1,1,-1,...,-1,1,-1,1,1,1,1,1,-1,-1
4,1,1,1,1,1,1,1,-1,1,-1,...,-1,-1,1,1,1,1,1,1,1,1


In [16]:
y = df.iloc[:,63:70]

y.head()

,63,64,65,66,67,68,69
0,1,-1,-1,-1,-1,-1,-1
1,-1,1,-1,-1,-1,-1,-1
2,-1,-1,1,-1,-1,-1,-1
3,-1,-1,-1,1,-1,-1,-1
4,-1,-1,-1,-1,1,-1,-1


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=7, stratify=y)

In [44]:
L0 = 63
L1 = 20
L2 = 7

learning_rate = 0.05
max_epochs = 1000

activation_func = Activations['tanh']
activation_deriv = D_Activations['tanh']

error_func = Errors['SR']
error_deriv = D_Errors['SR']

max_lim = 1
min_lim = -1

## treinando

In [49]:
weights_L1 = start_weights(L0, L1)
bias_L1 = start_bias(L1)
weights_L2 = start_weights(L1, L2)
bias_L2 = start_bias(L2)

fit(X_train, y_train)

## testando

In [53]:
y_ruido.style.background_gradient(cmap ='viridis')

,63,64,65,66,67,68,69
0,1,-1,-1,-1,-1,-1,-1
1,-1,1,-1,-1,-1,-1,-1
2,-1,-1,1,-1,-1,-1,-1
3,-1,-1,-1,1,-1,-1,-1
4,-1,-1,-1,-1,1,-1,-1
5,-1,-1,-1,-1,-1,1,-1
6,-1,-1,-1,-1,-1,-1,1
7,1,-1,-1,-1,-1,-1,-1
8,-1,1,-1,-1,-1,-1,-1
9,-1,-1,1,-1,-1,-1,-1


In [54]:
preds = predict(X_ruido)
preds.style.background_gradient(cmap ='viridis')

[ 0.99839738 -0.99999775 -0.99914511 -1.         -0.99999013 -0.99389764
 -0.99894457]
[-0.99999954  0.87238729 -1.          0.82840672 -1.         -0.99999563
 -1.        ]
[-0.96201607 -0.99658964  0.99981692 -1.         -0.99999991 -0.99974559
 -0.99978674]
[-0.99999365 -0.99346421 -1.          0.56217945 -0.99999925 -0.97591683
 -1.        ]
[-0.99999965 -0.95718441 -0.99999999 -0.9996469   0.86271555 -0.99951395
 -0.99960032]
[-0.99778384 -0.99999997 -0.9999994  -0.99839835 -0.99999999  0.57731783
 -0.99999215]
[-0.99992151 -0.99999715 -0.99999999 -0.9999999  -0.98749559 -0.99961538
  0.97337369]
[ 0.99913835 -0.99999899 -0.99668419 -1.         -0.99997023 -0.98914334
 -0.99877793]
[-0.99965941  0.81807774 -0.99725026 -0.99997275 -0.98171173 -0.99999459
 -0.99999948]
[-0.99859165 -0.99986324  0.99488351 -0.99999676 -0.99999985 -0.97888376
 -0.99917097]
[-1.         -0.91079056 -0.99999153  0.97479197 -1.         -0.99993499
 -0.99982967]
[-0.99999869  0.73281653  0.56935411 -0.999

,0,1,2,3,4,5,6
0,1,-1,-1,-1,-1,-1,-1
1,-1,1,-1,-1,-1,-1,-1
2,-1,-1,1,-1,-1,-1,-1
3,-1,-1,-1,1,-1,-1,-1
4,-1,-1,-1,-1,1,-1,-1
5,-1,-1,-1,-1,-1,1,-1
6,-1,-1,-1,-1,-1,-1,1
7,1,-1,-1,-1,-1,-1,-1
8,-1,1,-1,-1,-1,-1,-1
9,-1,-1,1,-1,-1,-1,-1


In [57]:
acertos = 0
erros = 0

for c1,c2 in zip(y_ruido.columns, preds.columns):
    for v1,v2 in zip(y_ruido[c1], preds[c2]):
        if v1 == 1:
            if v1 == v2:
                acertos += 1
            else:
                erros += 1
print(acertos, erros)

19 2
